In [1]:
#Function to generate a 3-panel plot for input arrays
def plot_array(dem, clim=None, titles=None, cmap='inferno', label=None, overlay=None, fn=None, close_fig=True):
    fig, ax = plt.subplots(1,1, sharex=True, sharey=True, figsize=(10,5))
    alpha = 1.0
    #Gray background
    ax.set_facecolor('0.5')
    #Force aspect ratio to match images
    ax.set(aspect='equal')
    #Turn off axes labels/ticks
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    if titles is not None:
        ax.set_title(titles[0])
    #Plot background shaded relief map
    if overlay is not None:
        alpha = 0.7
        ax.imshow(overlay, cmap='gray', clim=(1,255))
    #Plot each array
    im_list = [ax.imshow(dem, clim=clim, cmap=cmap, alpha=alpha)]
    fig.tight_layout()
    fig.colorbar(im_list[0], label=label, extend='both', shrink=0.5)
    if fn is not None:
        fig.savefig(fn, bbox_inches='tight', pad_inches=0, dpi=150)
    if close_fig:
        plt.close(fig)

In [2]:
#! /usr/bin/env python
"""
Process the various datasets that will be used for the debris thickness inversions
"""
import sys
import os
import re
import subprocess
from datetime import datetime, timedelta
import time
import pickle
from collections import OrderedDict

import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
from rasterio.merge import merge
from rasterio.warp import calculate_default_transform, reproject, Resampling
from scipy import ndimage
from scipy.optimize import curve_fit
from scipy.optimize import minimize
from scipy.stats import median_absolute_deviation
import xarray as xr
from osgeo import gdal, ogr, osr

from pygeotools.lib import malib, warplib, geolib, iolib, timelib


import debrisglobal.globaldebris_input as debris_prms
from debrisglobal.glacfeat import GlacFeat, create_glacfeat

calc_emergence=True

debug=False
extra_layers=True

csv_ending = '_mb_bins.csv'
outdir_csv = debris_prms.mb_binned_fp
if os.path.exists(outdir_csv) == False:
    os.makedirs(outdir_csv)

In [3]:
# Debris cover extent shapefile with statistics
dc_shp = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
dc_shp = dc_shp.sort_values(by=['RGIId'])

# Subset by percent debris-covered or debris-covered area
dc_shp_subset = dc_shp[((dc_shp['DC_Area__1'] > debris_prms.dc_percarea_threshold) | 
                        (dc_shp['DC_Area_v2'] / 1e6 > debris_prms.dc_area_threshold))
                        & (dc_shp['Area'] > debris_prms.min_glac_area)].copy()
dc_shp_subset.reset_index(inplace=True, drop=True)
dc_shp_subset['CenLon_360'] = dc_shp_subset['CenLon']
dc_shp_subset.loc[dc_shp_subset['CenLon_360'] < 0, 'CenLon_360'] = (
    360 + dc_shp_subset.loc[dc_shp_subset['CenLon_360'] < 0, 'CenLon_360'])

rgiid_list = [x.split('-')[1] for x in dc_shp_subset['RGIId'].values]
main_glac_rgi_subset = debris_prms.selectglaciersrgitable(rgiid_list)
    
main_glac_rgi_subset

663 glaciers in region 5 are included in this model run: ['00115', '00181', '00184', '00258', '00270', '00275', '00318', '00319', '00323', '00334', '00335', '00336', '00337', '00342', '00400', '00421', '00445', '00446', '00451', '00459', '00460', '00522', '00530', '00789', '00790', '00793', '00794', '00800', '00801', '00817', '00839', '00859', '00879', '00904', '00938', '01224', '01355', '01383', '01403', '01441', '01452', '01456', '01706', '01780', '02398', '02521', '02536', '02651', '02918', '03019'] and more
This study is focusing on 663 glaciers in region [5]


,O1Index,RGIId,CenLon,CenLat,O1Region,O2Region,Area,Zmin,Zmax,Zmed,Slope,Aspect,Lmax,Form,TermType,Surging,RefDate,glacno,rgino_str,RGIId_float
GlacNo,,,,,,,,,,,,,,,,,,,,
0,114,RGI60-05.00115,-52.1262,65.6000,5,1,72.377,143,1139,825,10.5,11,5472,0,0,9,20000823,115,05.00115,5.00115
1,180,RGI60-05.00181,-52.9288,65.7104,5,1,2.060,111,1015,640,22.2,1,2737,0,0,9,20000823,181,05.00181,5.00181
2,183,RGI60-05.00184,-52.9527,65.6998,5,1,2.807,241,925,625,18.9,283,2511,0,0,9,20000814,184,05.00184,5.00184
3,257,RGI60-05.00258,-52.5941,65.8403,5,1,4.232,378,1526,1010,21.2,303,3916,0,0,9,20000823,258,05.00258,5.00258
4,269,RGI60-05.00270,-52.5381,65.8321,5,1,5.630,351,1663,1062,17.3,231,5703,0,0,9,20000823,270,05.00270,5.00270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
658,20158,RGI60-05.20173,-21.0378,74.7637,5,1,3.393,827,1292,1062,11.3,125,1884,0,0,9,20010820,20173,05.20173,5.20173
659,20168,RGI60-05.20183,-22.6020,74.9837,5,1,3.712,1223,1402,1294,5.0,24,3386,0,0,9,20010820,20183,05.20183,5.20183
660,20183,RGI60-05.20198,-22.0165,74.9015,5,1,45.809,923,1649,1297,6.9,110,9041,1,0,9,20010820,20198,05.20198,5.20198


In [4]:
if os.path.exists(debris_prms.dhdt_vel_fns_fp) == False:
    os.makedirs(debris_prms.dhdt_vel_fns_fp)
dhdt_vel_fns_fn = debris_prms.dhdt_vel_fns_fn.replace('XXXX',debris_prms.roi)
if os.path.exists(debris_prms.dhdt_vel_fns_fp + dhdt_vel_fns_fn):
    dhdt_vel_fns_df = pd.read_csv(debris_prms.dhdt_vel_fns_fp + dhdt_vel_fns_fn)
else:
    dhdt_vel_fns_df = pd.DataFrame(np.zeros((main_glac_rgi_subset.shape[0], 3)),
                                   columns=['RGIId', 'dhdt_fullfn', 'vel_fullfn'])
    dhdt_vel_fns_df['RGIId'] = main_glac_rgi_subset['RGIId']

In [5]:
# np.where(main_glac_rgi_subset.rgino_str == '15.03473')

In [6]:
# ===== PROCESS EACH GLACIER =====
for nglac, glac_idx in enumerate(main_glac_rgi_subset.index.values):
# for nglac, glac_idx in enumerate([main_glac_rgi_subset.index.values[0]]):
# for nglac, glac_idx in enumerate([main_glac_rgi_subset.index.values[120]]): # Miage
# for nglac, glac_idx in enumerate([main_glac_rgi_subset.index.values[2307]]): # Ngozumpa

    glac_str = main_glac_rgi_subset.loc[glac_idx,'rgino_str']
    rgiid = main_glac_rgi_subset.loc[glac_idx,'RGIId']
    region = glac_str.split('.')[0]

    if int(region) < 10:
        glac_str_noleadzero = str(int(glac_str.split('.')[0])) + '.' + glac_str.split('.')[1]
    else:
        glac_str_noleadzero = glac_str

    if os.path.exists(debris_prms.hd_fp + debris_prms.hd_fn_sample.replace('XXXX',glac_str_noleadzero)) == False:

        print(nglac, glac_idx, rgiid)

        # Create glacier feature from ice thickness raster
        thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
        thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'
        
        if os.path.exists(thick_dir + thick_fn):
        
            gf = create_glacfeat(thick_dir, thick_fn)

            # Debris shape layer processing
            dc_shp_proj_fn = (debris_prms.glac_shp_proj_fp + glac_str + '_dc_crs' + 
                              str(gf.aea_srs.GetAttrValue("AUTHORITY", 1)) + '.shp')
            if os.path.exists(dc_shp_proj_fn) == False:
                dc_shp_init = gpd.read_file(debris_prms.debriscover_fp + debris_prms.debriscover_fn_dict[debris_prms.roi])
                dc_shp_single = dc_shp_init[dc_shp_init['RGIId'] == rgiid]
                dc_shp_single = dc_shp_single.reset_index()
                dc_shp_proj = dc_shp_single.to_crs({'init': 'epsg:' + str(gf.aea_srs.GetAttrValue("AUTHORITY", 1))})
                dc_shp_proj.to_file(dc_shp_proj_fn)
            dc_shp_ds = ogr.Open(dc_shp_proj_fn, 0)
            dc_shp_lyr = dc_shp_ds.GetLayer()


            # ==== CHECK IF TIF HAS DHDT DATA OVER THE GLACIER =====
            mb_fullfns = []
            find_mb = True
            dhdt_fn_wglacier = None
            for mb_fp in debris_prms.mb_fp_list_roi[debris_prms.roi]:
                if find_mb:
                    for i in os.listdir(mb_fp):
                        if i.endswith('.tif'):
                            mb_fullfns.append(mb_fp + i)
                    tif_count = 0
                    while find_mb and tif_count < len(mb_fullfns):
                        dhdt_fn = mb_fullfns[tif_count]
                        if debug:
                            print(tif_count, dhdt_fn.split('/')[-1])

                        # Add the filenames
                        fn_dict = OrderedDict()
                        # DEM
                        z1_fp = debris_prms.oggm_fp + 'dems/RGI60-' + str(region.zfill(2)) + '/'
                        z1_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_dem.tif'
                        fn_dict['z1'] = z1_fp + z1_fn
                        # Ice thickness
                        thick_dir = debris_prms.oggm_fp + 'thickness/RGI60-' + str(region.zfill(2)) + '/'
                        thick_fn = 'RGI60-' + str(region.zfill(2)) + '.' + rgiid.split('.')[1] + '_thickness.tif'
                        fn_dict['ice_thick'] = thick_dir + thick_fn
                        # dh/dt
                        fn_dict['dhdt'] = dhdt_fn
                        # ===== PROCESS THE DATA =====
                        #Expand extent to include buffered region around glacier polygon
                        warp_extent = geolib.pad_extent(gf.glac_geom_extent, width=debris_prms.buff_dist)
                        #Warp everything to common res/extent/proj
                        z1_gt = gdal.Open(fn_dict['z1']).GetGeoTransform()
                        z1_res = np.min([z1_gt[1], -z1_gt[5]])
                        ds_list = warplib.memwarp_multi_fn(fn_dict.values(), res=z1_res, extent=warp_extent, 
                                                           t_srs=gf.aea_srs, verbose=False, r='cubic')
                        ds_dict = dict(zip(fn_dict.keys(), ds_list))
                        gf.ds_dict = ds_dict

                        if 'z1' in ds_dict:
                            #This is False over glacier polygon surface, True elsewhere - can be applied directly
                            glac_geom_mask = geolib.geom2mask(gf.glac_geom, ds_dict['z1'])
                            glac_geom_mask_copy = glac_geom_mask.copy()
                            gf.z1 = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=glac_geom_mask)
                            # Debris cover
                            dc_shp_lyr_mask = geolib.lyr2mask(dc_shp_lyr, ds_dict['ice_thick'])
                            gf.dc_mask = np.ma.mask_or(dc_shp_lyr_mask, glac_geom_mask)
                            if 'dhdt' in ds_dict:
                                gf.dhdt = np.ma.array(iolib.ds_getma(ds_dict['dhdt']), mask=glac_geom_mask_copy)
                                gf.dhdt.mask = np.ma.mask_or(
                                    glac_geom_mask, np.ma.getmask(np.ma.masked_array(gf.dhdt.data, 
                                                                                     np.isnan(gf.dhdt.data))))
                                gf.dc_dhdt = np.ma.array(iolib.ds_getma(ds_dict['dhdt']), mask=glac_geom_mask_copy)
                                gf.dc_dhdt.mask = gf.dc_mask
                                gf.dc_area = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=gf.dc_mask)
                                # Count dhdt pixels
                                dc_dhdt_pixels = len(gf.dc_dhdt.nonzero()[0])
                                if dc_dhdt_pixels / gf.dc_area.count() * 100 > 75:
                                    dhdt_fn_wglacier = dhdt_fn
                                    find_mb = False
                                    if debug:
                                        print('\n# z1 dc pixels:', gf.dc_area.count())
                                        print('# dc_dhdt_pixels:', dc_dhdt_pixels)
                                        var_full2plot = gf.dhdt.copy()
                                        clim = malib.calcperc(var_full2plot, (2,98))
                                        plot_array(var_full2plot, clim, [glac_str + ' dhdt'], 'inferno', 'dhdt (m/yr)', 
                                                   close_fig=False)
                        # Loop over layers        
                        tif_count += 1

            # ==== CHECK IF VELOCITY DATA OVER THE GLACIER =====
            vel_fullfns = []
            find_vel = True
            vx_fn_wglacier = None
            if find_vel and dhdt_fn_wglacier is not None:
                vx_fns = debris_prms.vx_dir_dict_list[debris_prms.roi]
                tif_count = 0
                while find_vel and tif_count < len(vx_fns):
                    vx_fn = vx_fns[tif_count]

                    if debug:
                        print(tif_count, vx_fn.split('/')[-1])

                    # Add the filenames
                    # Velocity
                    fn_dict['vx'] = vx_fn
                    fn_dict['vy'] = vx_fn.replace('_vx', '_vy')
                    # ===== PROCESS THE DATA =====
                    ds_list = warplib.memwarp_multi_fn(fn_dict.values(), res=z1_res, extent=warp_extent, 
                                                       t_srs=gf.aea_srs, verbose=False, r='cubic')
                    ds_dict = dict(zip(fn_dict.keys(), ds_list))
                    gf.ds_dict = ds_dict
                    if 'vx' in ds_dict and 'vy' in ds_dict:
                        #Load surface velocity maps
                        gf.vx = np.ma.array(iolib.ds_getma(ds_dict['vx']), mask=glac_geom_mask)
                        gf.vy = np.ma.array(iolib.ds_getma(ds_dict['vy']), mask=glac_geom_mask)
                        gf.vm = np.ma.sqrt(gf.vx**2 + gf.vy**2)
                        gf.dc_vm = gf.vm.copy()
                        gf.dc_vm.mask = gf.dc_mask
                        # Count velocity pixels
                        dc_vel_pixels = len(gf.dc_vm.nonzero()[0])
                        if debug:
                                print('\n# z1 dc pixels:', gf.dc_area.count())
                                print('# dc vel_pixels:', dc_vel_pixels)
                                var_full2plot = gf.vm.copy()
                                clim = malib.calcperc(var_full2plot, (2,98))
                                plot_array(var_full2plot, clim, [glac_str + ' velocity'], 'inferno', 'vm (m/yr)', 
                                           close_fig=False)
                        if dc_vel_pixels / gf.dc_area.count() * 100 > 75:
                            vx_fn_wglacier = vx_fn
                            find_vel = False

                    # Loop over layers        
                    tif_count += 1


            # ===== Add layers =====
            if dhdt_fn_wglacier is not None and vx_fn_wglacier is not None:
                gf.add_layers(dc_shp_lyr, gf_add_dhdt=True, dhdt_fn=dhdt_fn_wglacier, gf_add_vel=True, vx_fn=vx_fn_wglacier, 
                              gf_add_ts=False, gf_add_slope_aspect=True, gf_add_ts_info=False, calc_emergence=True, 
                              debug_emergence=False)
                # Save dhdt and vel filenames
                dhdt_vel_fns_df.loc[glac_idx,:] = [rgiid, dhdt_fn_wglacier, vx_fn_wglacier]

                # ===== PLOTS =====
#                 plot_layers = True
                plot_layers = False
                if plot_layers:
                    # DEM
                    var_full2plot = gf.z1.copy()
                    clim = malib.calcperc(var_full2plot, (2,98))
                    plot_array(var_full2plot, clim, [glac_str + ' DEM'], 'inferno', 'elev (masl)', close_fig=False)
                    # Elevation change
                    var_full2plot = gf.dhdt.copy()
                    clim = malib.calcperc(var_full2plot, (2,98))
                    plot_array(var_full2plot, clim, [glac_str + ' dhdt'], 'inferno', 'dhdt (m/yr)', close_fig=False)
                    # Velocity
                    var_full2plot = gf.vm.copy()
                    clim = malib.calcperc(var_full2plot, (2,98))
                    plot_array(var_full2plot, clim, [glac_str + ' velocity'], 'inferno', 'vel (m/yr)', close_fig=False)
                    # Ice thickness
                    var_full2plot = gf.H.copy()
                    clim = malib.calcperc(var_full2plot, (2,98))
                    plot_array(var_full2plot, clim, [glac_str + ' ice thickness'], 'inferno', 'H (m)', close_fig=False)
                    # Emergence velocity
                    if gf.emvel is not None:
                        var_full2plot = gf.emvel.copy()
                        clim = malib.calcperc(var_full2plot, (2,98))
                        plot_array(var_full2plot, clim, [glac_str + ' emvel'], 'inferno', 'emvel (m/yr)', close_fig=False)
                    # Surface temperature
                    if gf.ts is not None:
                        var_full2plot = gf.ts.copy()
                        clim = malib.calcperc(var_full2plot, (2,98))
                        plot_array(var_full2plot, clim, [glac_str + ' Ts'], 'inferno', 'ts (degC)', close_fig=False)
                        
                # ===== Export debris thickness and melt factor maps ===== 
                export_layers = False
                if export_layers:
                    layers2export = [[gf.z1, glac_str + '-z1.tif'], 
                                     [gf.dhdt, glac_str + '-dhdt.tif'],
                                     [gf.vm, glac_str + '-vm.tif'],
                                     [gf.vx, glac_str + '-vx.tif'],
                                     [gf.vy, glac_str + '-vy.tif'],
                                     [gf.H, glac_str + '-H.tif']]
                    for layer2export in layers2export:
                        iolib.writeGTiff(layer2export[0], debris_prms.output_fp + layer2export[1], gf.ds_dict['z1'])

                # Bin data
                outbins_df, z_bin_edges = gf.hist_plot(bin_width=debris_prms.mb_bin_size)
                # Export binned data
                if int(gf.feat_fn.split('.')[0]) < 10:
                    outbins_fullfn = os.path.join(outdir_csv, gf.feat_fn[0:7] + csv_ending)
                else:
                    outbins_fullfn = os.path.join(outdir_csv, gf.feat_fn[0:8] + csv_ending)
                outbins_df.loc[:,:] = np.nan_to_num(outbins_df.loc[:,:],0)
                outbins_df.to_csv(outbins_fullfn, index=False)

# Save updated filenames
dhdt_vel_fns_df.to_csv(debris_prms.dhdt_vel_fns_fp + dhdt_vel_fns_fn, index=False)

0 0 RGI60-05.00115
1 1 RGI60-05.00181
2 2 RGI60-05.00184
3 3 RGI60-05.00258
4 4 RGI60-05.00270
5 5 RGI60-05.00275
6 6 RGI60-05.00318
7 7 RGI60-05.00319
8 8 RGI60-05.00323
9 9 RGI60-05.00334
10 10 RGI60-05.00335
11 11 RGI60-05.00336
12 12 RGI60-05.00337
13 13 RGI60-05.00342
14 14 RGI60-05.00400
15 15 RGI60-05.00421
16 16 RGI60-05.00445
17 17 RGI60-05.00446
18 18 RGI60-05.00451
19 19 RGI60-05.00459
20 20 RGI60-05.00460
21 21 RGI60-05.00522
22 22 RGI60-05.00530
23 23 RGI60-05.00789
24 24 RGI60-05.00790
25 25 RGI60-05.00793
26 26 RGI60-05.00794
27 27 RGI60-05.00800
28 28 RGI60-05.00801
29 29 RGI60-05.00817
30 30 RGI60-05.00839
31 31 RGI60-05.00859
32 32 RGI60-05.00879
33 33 RGI60-05.00904
34 34 RGI60-05.00938
35 35 RGI60-05.01224
36 36 RGI60-05.01355
37 37 RGI60-05.01383
38 38 RGI60-05.01403
39 39 RGI60-05.01441
40 40 RGI60-05.01452
41 41 RGI60-05.01456
42 42 RGI60-05.01706
43 43 RGI60-05.01780
44 44 RGI60-05.02398
45 45 RGI60-05.02521
46 46 RGI60-05.02536
47 47 RGI60-05.02651
48 48 RGI60-

366 366 RGI60-05.10574
367 367 RGI60-05.10575
368 368 RGI60-05.10583
369 369 RGI60-05.10584
370 370 RGI60-05.10586
371 371 RGI60-05.10587
372 372 RGI60-05.10590
373 373 RGI60-05.10593
374 374 RGI60-05.10598
375 375 RGI60-05.10599
376 376 RGI60-05.10603
377 377 RGI60-05.10609
378 378 RGI60-05.10610
379 379 RGI60-05.10611
380 380 RGI60-05.10617
381 381 RGI60-05.10623
382 382 RGI60-05.10628
383 383 RGI60-05.10629
384 384 RGI60-05.10636
385 385 RGI60-05.10640
386 386 RGI60-05.10668
387 387 RGI60-05.10678
388 388 RGI60-05.10679
389 389 RGI60-05.10680
390 390 RGI60-05.10699
391 391 RGI60-05.10702
392 392 RGI60-05.10704
393 393 RGI60-05.10710
394 394 RGI60-05.10733
395 395 RGI60-05.10735
396 396 RGI60-05.10747
397 397 RGI60-05.10748
398 398 RGI60-05.10749
399 399 RGI60-05.10750
400 400 RGI60-05.10754
401 401 RGI60-05.10756
402 402 RGI60-05.10763
403 403 RGI60-05.10766
404 404 RGI60-05.10769
405 405 RGI60-05.10775
406 406 RGI60-05.10831
407 407 RGI60-05.10833
408 408 RGI60-05.10834
409 409 RGI

/Users/davidrounce/anaconda3/envs/debris_thickness_global/lib/python3.6/site-packages/numpy/ma/core.py:1178: RuntimeWarning: invalid value encountered in multiply
  masked_da = umath.multiply(m, da)


525 525 RGI60-05.13471
526 526 RGI60-05.13499
527 527 RGI60-05.13505
528 528 RGI60-05.13508
529 529 RGI60-05.13512
530 530 RGI60-05.13536
531 531 RGI60-05.13567
532 532 RGI60-05.13568
533 533 RGI60-05.13575
534 534 RGI60-05.13663
535 535 RGI60-05.13666
536 536 RGI60-05.13667
537 537 RGI60-05.13676
538 538 RGI60-05.13710
539 539 RGI60-05.13717
540 540 RGI60-05.13753
541 541 RGI60-05.13754
542 542 RGI60-05.13774
543 543 RGI60-05.13812
544 544 RGI60-05.13862
545 545 RGI60-05.13864
546 546 RGI60-05.13876
547 547 RGI60-05.13878
548 548 RGI60-05.13893
549 549 RGI60-05.13895
550 550 RGI60-05.13906
551 551 RGI60-05.13910
552 552 RGI60-05.13916
553 553 RGI60-05.13918
554 554 RGI60-05.13930
555 555 RGI60-05.13931
556 556 RGI60-05.13938
557 557 RGI60-05.13949
558 558 RGI60-05.13950
559 559 RGI60-05.13953
560 560 RGI60-05.13961
561 561 RGI60-05.13965
562 562 RGI60-05.13967
563 563 RGI60-05.13974
564 564 RGI60-05.13983
565 565 RGI60-05.13985
566 566 RGI60-05.13995
567 567 RGI60-05.14094
568 568 RGI

In [7]:
print('\n\nDONE!\n\n')



DONE!




In [ ]:
# ===== SHEAN ESTIMATE OF FLUX DIVERGENCE QUICKLY ======
#                 if gf.H is not None:
#                     #Compute flux
#                     gf.Q = gf.H * debris_prms.v_col_f * np.array([gf.vx, gf.vy])
#                     #Note: np.gradient returns derivatives relative to axis number, so (y, x) in this case
#                     #Want x-derivative of x component
#                     gf.divQ = np.gradient(gf.Q[0])[1] + np.gradient(gf.Q[1])[0]
# #                     gf.divQ = gf.H*(np.gradient(v_col_f*gf.vx)[1] + np.gradient(v_col_f*gf.vy)[0]) \
# #                             + v_col_f*gf.vx*(np.gradient(gf.H)[1]) + v_col_f*gf.vy*(np.gradient(gf.H)[0])
#                     #Should smooth divQ, better handling of data gaps

In [ ]:
# ===== OLD CHECK DEM FOR ERRORS AND REPLACE SCRIPT (no longer needed with OGGM processing) =====
#         #Create buffer around glacier polygon
#         glac_geom_buff = gf.glac_geom.Buffer(debris_prms.buff_dist)
#         #This is False over glacier polygon surface, True elsewhere - can be applied directly
#         glac_geom_buff_mask = geolib.geom2mask(glac_geom_buff, ds_dict['ice_thick'])
        
#             # ds masks
#             ds_list_masked = [iolib.ds_getma(i) for i in ds_list]
#             dem1 = np.ma.masked_less_equal(ds_list_masked[0], 0)
#             dems_mask = dem1.mask
#             if verbose:
#                 print('list of datasets:', len(ds_list_masked), fn_dict.values())

#             #Combine to identify ~1 km buffer around glacier polygon over static rock
#             static_buffer_mask = np.ma.mask_or(~glac_shp_lyr_mask, glac_geom_buff_mask)
#             static_shp_lyr_mask = np.ma.mask_or(static_buffer_mask, dems_mask)
        
        
#             # Check if DEM has huge errors or not - replace if necessary
#             if input.roi in ['01']:

#                 gf.z1_check = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=glac_geom_mask)
#                 if gf.z1_check.min() < 0:

#                     # Add backup DEM for regions with known poor quality (ex. Alaska)
#                     print('switching DEMs')
#                     fn_dict['z1_backup'] = input.z1_backup_dict[input.roi]
#                     # Warp everything to common res/extent/proj (a second time)
#                     ds_list = warplib.memwarp_multi_fn(fn_dict.values(), res=z1_res, \
#                             extent=warp_extent, t_srs=aea_srs, verbose=verbose, \
#                             r='cubic')
#                     ds_dict = dict(zip(fn_dict.keys(), ds_list))

#                     if verbose:
#                         print(ds_list)
#                         print(fn_dict.keys())

#                     # ds masks
#                     ds_list_masked = [iolib.ds_getma(i) for i in ds_list]
#                     dem1 = np.ma.masked_less_equal(ds_list_masked[-1], 0)
#                     dems_mask = dem1.mask
#                     if verbose:
#                         print('list of datasets:', len(ds_list_masked), fn_dict.values())

#                     #Combine to identify ~1 km buffer around glacier polygon over static rock
#                     static_buffer_mask = np.ma.mask_or(~glac_shp_lyr_mask, glac_geom_buff_mask)
#                     static_shp_lyr_mask = np.ma.mask_or(static_buffer_mask, dems_mask)

#                     #This is False over glacier polygon surface, True elsewhere - can be applied directly
#                     glac_geom_mask = geolib.geom2mask(gf.glac_geom, ds_dict['z1_backup'])
#                     gf.z1 = np.ma.array(iolib.ds_getma(ds_dict['z1_backup']), mask=glac_geom_mask)
#                     #gf.z1 = np.ma.array(iolib.ds_getma(ds_dict['z1']), mask=glac_geom_mask)

#                     # Debris cover
#                     dc_mask = np.ma.mask_or(dc_shp_lyr_mask, glac_geom_mask)
#                     gf.dc_area = np.ma.array(iolib.ds_getma(ds_dict['z1_backup']), mask=dc_mask)